<a href="https://colab.research.google.com/github/TiagoIesbick/dashboard-etl/blob/main/empenhos_folha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
  <img src="https://www2.portoalegre.rs.gov.br/proweb3_geral/images/icones/brasao_pmpa_cor.png" width="120" />
</p>
<p align="center">
    EMPENHOS FOLHA
</p>
<br>
<p align="center">
		<em>Desenvolvido com os softwares e ferramentas abaixo.</em>
</p>
<p align="center">
    <img src="https://img.shields.io/badge/Python-3776AB.svg?style=flat&logo=Python&logoColor=white" alt="Python">
    <img src="https://img.shields.io/badge/Colab-F9AB00.svg?style=flat&logo=googlecolab&logoColor=525252" alt="Colab">
    <img src="https://img.shields.io/badge/Pandas-2C2D72.svg?style=flat&logo=pandas&logoColor=white" alt="Pandas">
    <img src="https://img.shields.io/badge/.ENV-ECD53F.svg?style=flat&logo=dotenv&logoColor=black" alt=".ENV">
</p>

---

## 🚀 Começando

***Requisito***

* **Conta Google**

### 🔧 Instalação

1. Com sua conta google ativada no navegador, visite o seguinte link:
<a href="https://drive.google.com/drive/folders/1Aw9xiSycB0-bCjHusFhhJpYRNKevqjKi?usp=sharing">
https://drive.google.com/drive/folders/1Aw9xiSycB0-bCjHusFhhJpYRNKevqjKi?usp=sharing</a>

2. Na pasta `Compartilhados comigo` de seu Google Drive:
  1.   Encontre a pasta `PARAMETRIZAÇÕES FOPAG`;
  2.   Cique nos três pontinhos ao lado direito da pasta;
  3.   Vá em `Organizar > Adicionar Atalho > Todos os locais > Meu Drive > Adicionar`.

---

## 🤖 Executando empenhos_folha.ipynb

1. Certifique-se de ter feito upload dos arquivos modificadores da folha em suas respectivas pastas e com seus nomes padronizados;
2. Execute este arquivo indo em `Ambiente de execução > Executar Tudo` ou digite `Ctrl+F9`;
3. Quando questionado, dê as **permissões** para o Colab;
4. Insira o número da empresa no campo solicitado;
```
     50: CENTRALIZADA
     51: DMLU
     52: DEMHAB
     55: DMAE
     58: PREVIMPA
     70: FASC
     80: CÂMARA
```
5. Insira o ANO desejado no campo solicitado <small>(deve possuir 4 dígitos e não pode ser maior do que o ano corrente)</small>;
6. Insira o número do MÊS no campo solicitado <small>(a união de ano e mês não pode se referir a um mês no futuro)</small>;
7. Insira o número do DIA da saída de recursos no campo solicitado <small>(não pode ser um dia de final de semana)</small>;
8. Insira o número do período da folha no campo solicitado:
```
     0: Outros
     1: Quinzenal
     2: Mensal
     3: Suplementar
     4: Natalina
```
9. Pode-se conferir a prévia dos empenhos SIGEF, baixando a planilha excel localizada na pasta virtual do Colab <small>(ícone de pasta ao lado esquerdo dessa tela)</small>;
10. Alternativamente, pode-se conferir a prévia dos empenhos SIGEF em seu e-mail de usuário;
11. Baixe os arquivos APP da pasta APPs do google drive e carregue-os para a pasta SFTP do SIGEF.

---
<p align="center">Desenvolvido com 💜 por Tiago Iesbick</p>

### Bibliotecas

In [1]:
%%capture
!pip install tabula-py[jpype]
from google.colab import drive
from google.colab import auth
from cryptography.fernet import Fernet
import pandas as pd
from datetime import datetime
import pytz
from pytz import timezone
import pathlib
import re
import smtplib
from email.message import EmailMessage
from functools import cache
from collections import namedtuple
from tabula import read_pdf
import subprocess, json, time, random

### Setup de Ambiente

In [3]:
%%capture
# montar drive
drive.mount('/content/drive')

# usuário
auth.authenticate_user()
user = !gcloud config get account

# opening the key
with open(r'/content/drive/MyDrive/PARAMETRIZAÇÕES FOPAG/utils/filekey.key', 'rb') as filekey:
    key = filekey.read()

# using the generated key
fernet = Fernet(key)

# opening the encrypted file
with open(r'/content/drive/MyDrive/PARAMETRIZAÇÕES FOPAG/utils/.env', 'rb') as enc_file:
    encrypted = enc_file.read()

# decrypting the file
decrypted = fernet.decrypt(encrypted)

# variáveis de ambiente
%env ${decrypted.decode('utf-8').split('\r\n')[0].replace("'", "")}
%env ${decrypted.decode('utf-8').split('\r\n')[1].replace("'", "")}

# deleting decrypted
del decrypted

### Seleção de Dados Ergon

In [4]:
def is_weekend(y: int,m: int,d: int) -> bool:
  try:
    return True if datetime(y,m,d).weekday() > 4 else False
  except:
    return True

def is_future(y: int, m: int, current_year: int, current_month: int) -> bool:
  try:
    return True if datetime(y,m,1) > datetime(current_year, current_month, 1) else False
  except:
    return True

e, y, m, d, p = 0, 0, 0, 0, -1

br = timezone('America/Sao_Paulo')
now = datetime.now(pytz.utc).astimezone(br)
t = now.strftime('%Y%m%d')
current_year = int(now.strftime('%Y'))
current_month = int(now.strftime('%m'))


while e not in [50, 51, 52, 55, 58, 70, 80]:
  e = int(input('Insira o número da EMPRESA: '))
while not bool(re.search(r'\d{4}', str(y)) and y <= current_year):
  y = int(input('Insira o ANO: '))
while is_future(y, m, current_year, current_month):
  m = int(input('Insira o número do MÊS: '))
while is_weekend(y,m,d) :
  d = int(input('''Insira o número do DIA em que o valor sairá da conta bancária:
  obs.: não são permitidos finais de semana.
   '''))
while not 0 <= p <= 4:
  p = int(input('Insira o número do PERÍODO da folha: '))

Insira o número da EMPRESA: 58
Insira o ANO: 2026
Insira o número do MÊS: 1
Insira o número do DIA em que o valor sairá da conta bancária:
  obs.: não são permitidos finais de semana.
   29
Insira o número do PERÍODO da folha: 2


### Utilidades

In [5]:
# lendo e tratando arquivo de cadastro de credores SIGEF
df_cnpj = pd.read_excel(
    r'/content/drive/MyDrive/PARAMETRIZAÇÕES FOPAG/utils/Cadastro_Credores/Cadastros CNPJ Credor FOPAG.xlsx',
    header=1,
    converters={
        'Banco': str, 'Agência': str, 'Conta': str, 'Processo': str, 'Bancoorigem': str,
        'Agênciaorigem': str, 'Contaorigem': str, 'Processo': str, 'Complemento': str
    }).rename(
        columns={
            'U. Orç': 'unidorc', 'Agência': 'agencia', 'Agênciaorigem': 'agenciaorigem'
        }).rename(columns=str.lower)
df_cnpj.fillna(value='0', inplace=True)
df_cnpj['cnpj'] = df_cnpj['cnpj'].str.replace(r'\D', '', regex=True).str.zfill(14)
df_cnpj['banco'] = df_cnpj['banco'].str.zfill(3)
df_cnpj['agencia'] = df_cnpj['agencia'].str.zfill(7)
df_cnpj['conta'] = df_cnpj['conta'].str.replace(r'\D', '', regex=True).str.zfill(10)
df_cnpj['bancoorigem'] = df_cnpj['bancoorigem'].str.zfill(3)
df_cnpj['agenciaorigem'] = df_cnpj['agenciaorigem'].str.zfill(6)
df_cnpj['contaorigem'] = df_cnpj['contaorigem'].str.replace(r'\D', '', regex=True).str.zfill(10)
df_cnpj['processo'] = df_cnpj['processo'].str.replace(r'\D', '', regex=True).str.zfill(20)
df_cnpj['complemento'] = df_cnpj['complemento'].str.zfill(5)
df_cnpj.drop_duplicates('unidorc', inplace=True, ignore_index=True)

# lendo parâmetros empenháveis
df_parm = pd.read_excel(
    r'/content/drive/MyDrive/PARAMETRIZAÇÕES FOPAG/utils/Parâmetros/parametros_empenhaveis.xlsx',
    converters={'natureza_despesa': str}
)

# dicionários úteis
dict_p = {0: 'OUTROS', 1: 'QUINZENAL', 2: 'MENSAL', 3: 'SUPLEMENTAR', 4: 'NATALINA'}
dict_m = {1: 'JANEIRO', 2: 'FEVEREIRO', 3: 'MARCO', 4: 'ABRIL', 5: 'MAIO', 6: 'JUNHO',
          7: 'JULHO', 8: 'AGOSTO', 9: 'SETEMBRO', 10: 'OUTUBRO', 11: 'NOVEMBRO', 12: 'DEZEMBRO'}
dict_empresa = {50: 'PMPA', 51: 'DMLU', 52: 'DHAB', 55: 'DMAE', 58: 'PREV', 70: 'FASC', 80: 'CMPA'}

# assunto do e-mail
subject = rf'Prévia Empenhos Folha {dict_p[p]} {dict_m[m]}/{y}'

# conteúdo do e-mail
email_content = rf'''Prezado(a),

Em anexo, segue planilha para conferência da prévia dos empenhos da folha {dict_p[p]} {dict_m[m]}/{y}.

Cordialmente,
Tiago Iesbick.
'''

### Classes

In [ ]:
class ReadErgonAPI:

  e, y, m, d, p, t = e, y, m, d, p, t


  def __init__(self):
    self.df = pd.DataFrame()


  def read_ergon_api(self) -> pd.DataFrame:
    tipos = [1, 2, 10, 11]
    if self.e == 58:
      tipos = tipos + [*range(12, 40, 1)]
    for i in tipos:
      api_ergon = (
          rf"https://ergonrest.procempa.com.br/ergonrest/api/sdo"
          rf"?mesFolha={self.m}&anoFolha={self.y}"
          rf"&empresa={self.e}&tipo={i}&periodo={self.p}"
      )

      # pd.read_json no python 3.12 deixou de ser compatível com a api do ergon.
      # Usaremos curl para contornar essa incompatibilidade.
      # Erro '35: OpenSSL SSL_connect: Connection reset by peer in connection to ergonrest.procempa.com.br:443' passou a acontecer às vezes.
      # Tentaremos até 3 vezes, quando ocorrerem erros.
      # Usaremos um TLS de renegociação diferente, caso em 3 tentativas o erro persista.
      # Tal configuração reduziu os erros para virtualmente zero.
      # Ficar atento aos eventuais erros impressos nas células de saída.

      # Try TLS 1.2 first (more stable for older servers)
      for tls in ["--tlsv1.2", "--tlsv1.3"]:
        cmd = [
            "curl", "-s", "--show-error", "--fail",
            "--retry", "3",
            "--retry-delay", "2",
            "--retry-all-errors",
            tls, "--noproxy", "*", "--location",
            "--max-time", "90", api_ergon
        ]

        result = subprocess.run(cmd, capture_output=True, text=True)
        if result.returncode == 0 and result.stdout.strip():
          break

      if result.returncode != 0 or not result.stdout.strip():
        error_msg = result.stderr.strip() or result.stdout.strip() or "<no message>"
        print(f"[ERRO tipo={i}] código {result.returncode}: {error_msg}")
        time.sleep(1)
        continue

      try:
        data = json.loads(result.stdout)
        df = pd.DataFrame(data)
        self.df = pd.concat([self.df, df], ignore_index=True)
      except json.JSONDecodeError as exc:
        print(f"[ERRO tipo={i}] JSON inválido: {exc}")
      time.sleep(0.3 + random.random() * 0.5)

    return self.df


  def clean_data_ergon_api(self) -> pd.DataFrame:
    if not self.df.empty:
      # deletando colunas que não serão usadas
      self.df.drop(['mesAnoFolha', 'numFolha', 'periodo', 'geracao', 'ehCons', 'tipoapos', 'tipo', 'codcont', 'nomeRubrica', 'nmconsig',
                    'regimejur', 'tipovinc', 'categoria', 'comissao', 'mesFolha', 'anoFolha', 'vincOrc'], axis=1, inplace=True)
      # passando colunas unidorc e projativ para inteiro
      self.df[['unidorc', 'projativ']] = self.df[['unidorc', 'projativ']].astype(int)
      # corrigindo as rubricas 231, 289, 290, 548, 573 para NÃO empenháveis.
      self.df.loc[self.df.rubrica.isin([231, 289, 290, 548, 573]), 'tpemp'] = 'N'
      # corrigindo as rubricas 144 e 467 para Empenháveis
      self.df.loc[self.df.rubrica.isin([144, 467]), 'tpemp'] = 'S'

      # corrigindo registros da centralizada
      if self.e == 50:
        # corrigindo unidade orçamentária e projeto/atividade das rubricas 331 e 332 que estejam fora das unidades 1801 e 1804.
        self.df.loc[(self.df.rubrica.isin([331, 332])) &
                    (~self.df.unidorc.isin([1801, 1804])), ['unidorc', 'projativ']] = 1804, 4005
        # corrigindo projeto/atividade das unidade orçamentárias 8601 e 8701 para projeto/atividade 4501 e 4540
        self.df.loc[self.df.unidorc.isin([8601]), ['projativ']] = 4501
        self.df.loc[self.df.unidorc.isin([8701]), ['projativ']] = 4540

      # corrigindo registros do PREVIMPA
      if self.e == 58:
        # correção rubrica 182, lançada em unidade orçamentária e projeto / atividade zerados
        self.df.loc[(self.df.rubrica == 182) & (self.df.unidorc == 0) & (self.df.projativ == 0), ['unidorc', 'projativ']] = 7001, 2870
        # deletando unidades estranhas ao PREVIMPA, fruto de salário família (rubrica 4) e
        # contracheques zerados ou negativos
        self.df = self.df[self.df.unidorc.isin([7001, 7002, 7003])].reset_index(drop=True)
        # corrigindo regprevgeral do dataframe de inativos para 2-RPPS
        self.df.loc[self.df.unidorc.isin([7002, 7003]), 'regprevgeral'] = '2-RPPS'
        # corrigindo rubricas de vantagens lançadas como desconto em contracheques zerados ou negativos da folha de inativos para NÃO empenháveis
        self.df.loc[(self.df.unidorc.isin([7002, 7003])) &
                    (self.df.descVant == 2) &
                    (self.df.rubrica.isin(
                        [*range(4,94,1)] + [95, 96] + [*range(98,144,1)] + [*range(145, 400, 1)]
                    )), 'tpemp'] = 'N'
        # corrigindo rubrica 996 para NÃO empenhável dentro da folha de inativos
        self.df.loc[(self.df.unidorc.isin([7002, 7003])) &
                    (self.df.rubrica == 996), 'tpemp'] = 'N'
        # definindo as situações: 0 - folha ATIVOS; 1 - folha APOSENTADOS; 2 - folha PENSIONISTAS
        self.df.loc[self.df.unidorc == 7001, 'situacao'] = 0
        self.df.loc[(self.df.unidorc.isin([7002, 7003])) &
                    (self.df.situacao == 'ATIVO'), 'situacao'] = 0
        self.df.loc[(self.df.unidorc.isin([7002, 7003])) &
                    (self.df.situacao == 'INATIVO'), 'situacao'] = 1
        self.df.loc[(self.df.unidorc.isin([7002, 7003])) &
                    (self.df.situacao == 'PENSIONISTA'), 'situacao'] = 2
        # corrigindo projetos/atividades da FASC 2738 e 2762 para 2740 e 2764 respectivamente
        self.df.loc[(self.df.unidorc == 7002) &
                    (self.df.projativ == 2738), 'projativ'] = 2740
        self.df.loc[(self.df.unidorc == 7003) &
                    (self.df.projativ == 2762), 'projativ'] = 2764

      # corrigindo registros da Câmara
      if self.e == 80:
        self.df[['projativ', 'unidorc']] = 2001, 100

      # definindo a folha ATIVOS (situacao = 0) das empresas, com exceção do PREVIMPA
      if self.e != 58:
        self.df.situacao = 0
    return self.df


  def select_budget_data(self) -> pd.DataFrame:
    df = pd.DataFrame()
    if not self.df.empty:
      df = self.df.loc[self.df.tpemp == 'S'].copy()
      if not df.empty:
        df.loc[df['descVant'] == 2, 'valor'] *= -1
        df.reset_index(drop=True, inplace=True)
    return df


  def select_extra_budget_data(self) -> pd.DataFrame:
    df = pd.DataFrame()
    if not self.df.empty:
      df = self.df.loc[self.df.tpemp != 'S'].copy()
      if not df.empty:
        df.reset_index(drop=True, inplace=True)
    return df


In [ ]:
class Payroll(ReadErgonAPI):

  df_cnpj = df_cnpj
  df_parm = df_parm

  def __init__(self, df: pd.DataFrame):
    self.df = df.copy()


  # pegar natureza da despesa
  def get_natureza_despesa(self) -> pd.DataFrame:
    self.df = self.df.merge(self.df_parm, on=['rubrica', 'regprevgeral', 'situacao'], how='left')
    return self.df


  #pegar subação
  def get_subacao(self) -> pd.DataFrame:
    # transformação de projeto/atividade em subação
    @cache
    def subacao(row: tuple) -> int:
      if row.projativ == 2870:
        if row.empresa == 50:
          match row.unidorc:
            case 2100:
              return 4390
            case 201:
              return 4385
            case 301:
              return 4386
            case 7501:
              return 4399
            case 8201:
              return 4405
            case 1001:
              return 4388
            case 7401:
              return 4398
            case 7301:
              return 4397
            case 8401:
              return 4456
            case 1301:
              return 4389
            case 8001:
              return 4404
            case 8501:
              return 4408
            case 8301:
              return 4406
            case 7701:
              return 4401
            case 7901:
              return 4403
            case 7801:
              return 4402
            case 801:
              return 4387
            case 7601:
              return 4400
            case 8101:
              return 4407
            case default:
              return row.projativ
        elif row.empresa == 51:
          return 4393
        elif row.empresa == 52:
          return 4391
        elif row.empresa == 55:
          return 4392
        elif row.empresa == 58:
          return 4396
        elif row.empresa == 70:
          return 4394
      else:
        return row.projativ
    self.df['subacao'] = list(self.df[['empresa', 'unidorc', 'projativ']].itertuples(index=False))
    self.df.subacao = self.df.subacao.map(subacao)
    return self.df


  # pegar fonte de recursos
  def get_fonte(self) -> pd.DataFrame:
    @cache
    def vinc(row: tuple) -> str:
      if row.empresa == 50:
        if 1500 <= row.unidorc <= 1502:
          return '1.540.031.001'
        elif row.unidorc == 1804:
          if row.rubrica in [187, 387]:
            return '1.605.504.001'
          else:
            return '1.500.001.040'
        else:
          return '1.500.001.001'
      elif row.empresa == 58:
        if row.unidorc == 7001:
          return '1.802.069.001'
        elif row.unidorc == 7002:
          return '1.801.049.001'
        elif row.unidorc == 7003:
          return '1.800.050.001'
      elif row.empresa in [52, 70, 80]:
        return '1.500.001.001'
      elif row.empresa == 51:
        return '1.753.140.001'
      else:
        return '1.501.400.001'
    self.df['fonte'] = list(self.df[['empresa', 'unidorc', 'rubrica']].itertuples(index=False))
    self.df.fonte = self.df.fonte.map(vinc)
    return self.df


  # máscara fonte de recursos
  def font_mask(self) -> pd.DataFrame:
    @cache
    def str_fonte(fonte: str) -> str:
      s = re.sub(r'\D', '', str(fonte))
      s = s[:1] + '.' + s[1:4] + '.' + s[4:7] + '.' + s[7:]
      return s
    if not hasattr(self.df, 'fonte'):
      self.fonte()
    self.df.fonte = self.df.fonte.map(str_fonte)
    return self.df


  # transformação de valores para formato sigef
  def sigef_value(self) -> pd.DataFrame:
    def float_to_int(value: float) -> int:
      if isinstance(value, float):
        return int(round(value * 100, 2))
      return value
    self.df.valor = self.df.valor.map(float_to_int)
    return self.df


  # reversão do formato sigef dos valores
  def revert_sigef_value(self) -> pd.DataFrame:
    def int_to_float(value: int) -> float:
      if isinstance(value, int):
        return float(value / 100)
      return value
    self.df.valor = self.df.valor.map(int_to_float)
    return self.df


  def add_data(self, df: pd.DataFrame) -> pd.DataFrame:
    if not df.empty:
      self.df = pd.concat([self.df, df], ignore_index=True)
    return self.df


  def group_previous_data(self) -> pd.DataFrame:
    self.df = self.df[['empresa', 'unidorc', 'subacao', 'natureza_despesa', 'regprevgeral', 'fonte', 'valor']].groupby(
                      by=['empresa', 'unidorc', 'subacao', 'natureza_despesa', 'fonte'], as_index=False
                      ).agg({'regprevgeral': lambda x: x.unique().tolist(), 'valor': 'sum'})
    return self.df


  def group_final_data(self) -> pd.DataFrame:
    self.df = self.df[['empresa', 'unidorc', 'subacao', 'natureza_despesa', 'fonte', 'valor']].groupby(
                      by=['empresa', 'unidorc', 'subacao', 'natureza_despesa', 'fonte'], as_index=False
                      ).sum()
    return self.df


  def group_by_element(self, df: pd.DataFrame) -> pd.DataFrame:
    if not df.empty:
      df['elemento'] = df['natureza_despesa'].str[:6]
      df = df[['empresa', 'unidorc', 'subacao', 'elemento', 'fonte', 'valor']].groupby(by=['empresa', 'unidorc', 'subacao', 'elemento', 'fonte'], as_index=False).sum()
    return df


  def permissive_discount(self, df: pd.DataFrame) -> pd.DataFrame:
    if not df.empty:
      for i in df.index:
        allocation = self.df.loc[(self.df.unidorc == df.loc[i, 'unidorc']) &
                                 (self.df.subacao == df.loc[i, 'subacao']) &
                                 (self.df.empresa == df.loc[i, 'empresa']) &
                                 (self.df.natureza_despesa == df.loc[i, 'natureza_despesa']) &
                                 (self.df.fonte == df.loc[i, 'fonte'])]
        if allocation['valor'].sum() == abs(df.loc[i, 'valor']):
          self.df.drop(allocation.index, inplace=True)
        else:
          self.df.loc[(self.df.unidorc == df.loc[i, 'unidorc']) &
                      (self.df.subacao == df.loc[i, 'subacao']) &
                      (self.df.empresa == df.loc[i, 'empresa']) &
                      (self.df.natureza_despesa == df.loc[i, 'natureza_despesa']) &
                      (self.df.fonte == df.loc[i, 'fonte']), 'valor'] -= abs(df.loc[i, 'valor'])
    return self.df


  def restrictive_discount(self, df: pd.DataFrame) -> pd.DataFrame:
    if not df.empty:
      for i in df.index:
        allocation = self.df.loc[(self.df.unidorc == df.loc[i, 'unidorc']) &
                                 (self.df.subacao == df.loc[i, 'subacao']) &
                                 (self.df.empresa == df.loc[i, 'empresa']) &
                                 (self.df.natureza_despesa == df.loc[i, 'natureza_despesa']) &
                                 (self.df.fonte == df.loc[i, 'fonte'])]
        if abs(df.loc[i, 'valor']) <= allocation['valor'].sum():
          self.permissive_discount(df.loc[i,:].to_frame().T)
        else:
          raise Exception(f'''O valor de {df.loc[i, 'valor']/100:,.2f} é superior ao valor de {allocation['valor'].sum()/100:,.2f} empenhado previamente na respectiva dotação:
          {allocation.to_string(index=False)}''')
    return self.df


  def change_fonte_verba_sus(self, df: pd.DataFrame) -> pd.DataFrame:
    if not df.empty:
      for i in df.index:
        self.df.loc[(self.df.unidorc == df.loc[i, 'unidorc']) &
                    (self.df.subacao == df.loc[i, 'subacao']) &
                    (self.df.empresa == df.loc[i, 'empresa']) &
                    (self.df.natureza_despesa == df.loc[i, 'natureza_despesa']), ['fonte', 'naoAlterar']] = df.loc[i, 'fonte'], 1
    return self.df


  def get_negative_values(self) -> pd.DataFrame:
    df = self.df.loc[self.df.valor <= 0]
    self.df.drop(df.index, inplace=True)
    return df


  def discount_negative_values(self) -> pd.DataFrame:
    df = self.get_negative_values()
    if not df.empty:
      for i in df.index:
        allocation = self.df.loc[(self.df.unidorc == df.loc[i, 'unidorc']) &
                                 (self.df.subacao == df.loc[i, 'subacao']) &
                                 (self.df.empresa == df.loc[i, 'empresa']) &
                                 (self.df.fonte == df.loc[i, 'fonte'])]
        if df.loc[i, 'unidorc'] not in [7002, 7003]:
          desc_nat = None
          if '2-RPPS' in df.loc[i, 'regprevgeral']:
            desc_nat = '31901101'
          else:
            desc_nat = '31901102'
          el = '319011'
          if desc_nat in allocation['natureza_despesa'].values and \
          allocation.loc[allocation['natureza_despesa'] == desc_nat, 'valor'].sum() >= abs(df.loc[i, 'valor']):
            df.loc[i, 'natureza_despesa'] = desc_nat
            self.permissive_discount(df.loc[i,:].to_frame().T)
          elif abs(df.loc[i, 'valor']) <= allocation.loc[allocation['natureza_despesa'].str[:6] == el, 'valor'].sum():
            self.discount_element(i, df, el, pd.DataFrame(), 'negative_values')
          else:
            raise Exception(f'''O valor de {df.loc[i, 'valor']/100:,.2f} não foi descontado,
            pois o valor de {allocation.loc[allocation['natureza_despesa'].str[:6] == el, 'valor'].sum()/100:,.2f} disponível no elemento {el} era insuficiente.''')
        else:
          el = df.loc[i, 'natureza_despesa'][:6]
          if abs(df.loc[i, 'valor']) <= allocation.loc[allocation['natureza_despesa'].str[:6] == el, 'valor'].sum():
            self.discount_element(i, df, el, pd.DataFrame(), 'negative_values')
          else:
            raise Exception(f'''O valor de {df.loc[i, 'valor']/100:,.2f} não foi descontado,
            pois o valor de {allocation.loc[allocation['natureza_despesa'].str[:6] == el, 'valor'].sum()/100:,.2f} disponível no elemento {el} era insuficiente.''')
    return self.df


  def discount_element(self, idx: int, df: pd.DataFrame, el: str, add_df: pd.DataFrame, kind: str) -> pd.DataFrame:
    l = len(el)
    allocation = self.df.loc[(self.df.unidorc == df.loc[idx, 'unidorc']) &
                             (self.df.subacao == df.loc[idx, 'subacao']) &
                             (self.df.empresa == df.loc[idx, 'empresa']) &
                             (self.df.natureza_despesa.str[:l] == el)]
    if kind != 'decreto':
      li = list(allocation.loc[allocation['fonte'] == df.loc[idx, 'fonte'], 'valor'].sort_values(ascending=False).items())
    elif l == 8:
      li = list(allocation['valor'].sort_values(ascending=False).items())
    elif l == 6:
      li = list(allocation.loc[allocation['naoAlterar'].isna(), 'valor'].sort_values(ascending=False).items())
    v = abs(df.loc[idx, 'valor']) * -1
    a = 0
    while v < 0 and a < len(li):
      if abs(v) >= li[a][1]:
        if kind == 'decreto':
          df_c = self.df.loc[li[a][0], :].copy()
          df_c['fonte'] = df.loc[idx, 'fonte']
          add_df = pd.concat([add_df, df_c.to_frame().T], ignore_index=True)
        self.df.drop(li[a][0], inplace=True)
      else:
        if kind == 'decreto':
          df_c = self.df.loc[li[a][0], :].copy()
          df_c['valor'] = abs(v)
          df_c['fonte'] = df.loc[idx, 'fonte']
          add_df = pd.concat([add_df, df_c.to_frame().T], ignore_index=True)
        self.df.loc[li[a][0], 'valor'] += v
      v += li[a][1]
      a += 1
    if kind == 'decreto' and not add_df.empty:
      self.add_data(add_df)
    return self.df


  def implement_decree(self, df: pd.DataFrame) -> pd.DataFrame:
    if not df.empty:
      self.df.loc[(self.df.unidorc == 1804) &
                  (self.df.fonte == '1.605.504.001') &
                  (self.df.natureza_despesa.isin(['31901157', '31901156', '31901129', '31901130'])), 'naoAlterar'] = 1
      for i in df.index:
        el = df.loc[i, 'elemento ou natureza de despesa']
        l = len(el)
        allocation = self.df.loc[(self.df.unidorc == df.loc[i, 'unidorc']) &
                                 (self.df.subacao == df.loc[i, 'subacao']) &
                                 (self.df.empresa == df.loc[i, 'empresa']) &
                                 (self.df.natureza_despesa.str[:l] == el)]
        if l == 8 and df.loc[i, 'valor'] <= allocation['valor'].sum():
          self.discount_element(i, df, el, pd.DataFrame(), 'decreto')
        elif l == 6 and df.loc[i, 'valor'] <= allocation.loc[allocation['naoAlterar'].isna(), 'valor'].sum():
          self.discount_element(i, df, el, pd.DataFrame(), 'decreto')
        else:
          raise Exception(f'''Decreto não efetivado, pois dotação foi informada erroneamente,
          ou valor de {df.loc[i, 'valor']/100:,.2f} contido no decreto era superior ao empenhado na respectiva dotação {allocation['valor'].sum()/100:,.2f}:
          {allocation.to_string(index=False)}''')
    return self.df


  # validar unidade orçamentária
  def validate_unidorc(self) -> None:
    ex = sorted(self.df.loc[~self.df.unidorc.isin(self.df_cnpj.unidorc), 'unidorc'].unique())
    l = len(ex)
    if l > 0:
      a = lambda a: 'á' if a == 1 else 'ão'
      raise Exception(f"A{'s'[:l^1]} seguinte{'s'[:l^1]} unidade{'s'[:l^1]} orçamentária{'s'[:l^1]} não est{a(l)} cadastrada{'s'[:l^1]} na planilha cadastro de credores: {str(ex)[1:-1]}.")


  # validar natureza da despesa
  def validate_natureza_despesa(self) -> None:
    if not hasattr(self.df, 'natureza_despesa'):
      self.get_natureza_despesa()
    ex = self.df.loc[self.df.natureza_despesa.isna(), ['rubrica', 'regprevgeral', 'situacao']].drop_duplicates(
        ['rubrica', 'regprevgeral', 'situacao']).sort_values(by=['rubrica', 'regprevgeral', 'situacao'])
    l = len(ex)
    if l > 0:
      a = lambda a: 'ão' if a == 1 else 'ões'
      raise Exception(f'''A{'s'[:l^1]} seguinte{'s'[:l^1]} combinaç{a(l)} não consta{'m'[:l^1]} na parametrização:
      {ex.to_string(index=False)}''')


In [ ]:
class ReadModifier(Payroll, ReadErgonAPI):


  def __init__(self):
    self.df = pd.DataFrame()
    self.Validation = namedtuple(
        'Validation', ['inicial_idx_name', 'last_idx_name', 'inicial_idx_date', 'last_idx_date', 'name', 'natureza_despesa',
                       'inicial_idx_empresa', 'last_idx_empresa', 'p'])
    self.dict_emp = {'PMPA': 50, 'DHAB': 52, 'PREV': 58, 'DMAE': 55, 'DMLU': 51, 'FASC': 70, 'CMPA': 80}
    self.dict_projativ = {'DHAB': '310102870', 'PREV': '700102870', 'DMAE': '400002870', 'DMLU': '500102870', 'FASC': '600102870'}
    self.dp_rub4 = {2740: {'unidorc': 2100, 'projativ': 2870, 'fonte': '1.500.001.001', 'natureza_despesa': '33900811'},
                    2742: {'unidorc': 1801, 'projativ': 4313, 'fonte': '1.500.001.001', 'natureza_despesa': '33900811'},
                    2744: {'unidorc': 1502, 'projativ': 2933, 'fonte': '1.500.001.001', 'natureza_despesa': '33900811'},
                    2747: {'unidorc': 1502, 'projativ': 2933, 'fonte': '1.500.001.001', 'natureza_despesa': '33900811'},
                    2749: {'unidorc': 1502, 'projativ': 2933, 'fonte': '1.500.001.001', 'natureza_despesa': '33900811'},
                    2751: {'unidorc': 1502, 'projativ': 2933, 'fonte': '1.500.001.001', 'natureza_despesa': '33900811'},
                    2752: {'unidorc': 3101, 'projativ': 2870, 'fonte': '1.500.001.001', 'natureza_despesa': '33900809'},
                    2754: {'unidorc': 4000, 'projativ': 2870, 'fonte': '1.501.400.001', 'natureza_despesa': '33900811'},
                    2756: {'unidorc': 5001, 'projativ': 2870, 'fonte': '1.753.140.001', 'natureza_despesa': '33900811'}}
    self.dp_empr = {1: 50, 2: 55, 3: 51, 4: 52, 5: 70, 6: 58, 7: 80}


  def validate_file(self, validation: tuple, file_path: pathlib.PosixPath) -> bool:
    if self.p == validation.p and file_path.is_file() and file_path.name[validation.inicial_idx_name:validation.last_idx_name] == validation.name and \
    file_path.name[validation.inicial_idx_date:validation.last_idx_date] == str(self.y) + str(self.m).zfill(2):
      return True
    return False


  def validate_p(self, file_name: str) -> None | int:
    return None if not bool(re.search(r'p(\d+)\.', file_name)) else int(re.findall(r'p(\d+)\.', file_name)[0])


  def read_folders_1111_1112(self) -> pd.DataFrame:
    @cache
    def vinc_aux(row: tuple) -> str:
      if row.empresa == 50:
        if row.PROJETO_ATIVIDADE[:4] == '1804':
          return '1.500.001.040'
        elif 1500 <= int(row.PROJETO_ATIVIDADE[:4]) <= 1502:
          return '1.540.031.001'
        else:
          return '1.500.001.001'
      elif row.empresa in [52, 70]:
        return '1.500.001.001'
      elif row.empresa == 58:
        return '1.802.069.001'
      elif row.empresa == 51:
        return '1.753.140.001'
      else:
        return '1.501.400.001'
    def validate_fonte(df: pd.DataFrame) -> None:
      if not df.empty and df['fonte'].isna().any():
        raise Exception(f'''As seguintes situações não estão previstas na função vinc_aux:
        {df.loc[df['fonte'].isna(), ['empresa', 'unidorc', 'projativ']].drop_duplicates().to_string(index=False)}''')
    folders = [pathlib.Path(r'/content/drive/MyDrive/PARAMETRIZAÇÕES FOPAG/modificadores_folha/Poa1111'),
               pathlib.Path(r'/content/drive/MyDrive/PARAMETRIZAÇÕES FOPAG/modificadores_folha/Poa1112')]
    validation = {'Poa1111': self.Validation(3, 7, 17, 23, '1111', '31901145', 12, 16, 2),
                  'Poa1112': self.Validation(3, 7, 17, 23, '1112', '31901146', 12, 16, 2)}
    for folder in folders:
      for arquivo in folder.iterdir():
        if self.validate_file(validation[folder.parts[-1]], arquivo) and \
        self.dict_emp[arquivo.name[validation[folder.parts[-1]].inicial_idx_empresa:validation[folder.parts[-1]].last_idx_empresa]] == self.e:
          df = pd.read_excel(arquivo, converters={'PROJETO_ATIVIDADE': str})
          if df['VALOR'].isna().any():
            pathlib.Path.unlink(arquivo)
            continue
          df['natureza_despesa'] = validation[folder.parts[-1]].natureza_despesa
          df['empresa'] = self.e
          if arquivo.name[validation[folder.parts[-1]].inicial_idx_empresa:validation[folder.parts[-1]].last_idx_empresa] != 'PMPA':
            df['PROJETO_ATIVIDADE'] = self.dict_projativ[arquivo.name[validation[folder.parts[-1]].inicial_idx_empresa:validation[folder.parts[-1]].last_idx_empresa]]
          df['regprevgeral'] = '2-RPPS'
          df['unidorc'] = df['PROJETO_ATIVIDADE'].str[:4].astype(int)
          df['projativ'] = df['PROJETO_ATIVIDADE'].str[-4:].astype(int)
          df['fonte'] = list(df[['empresa', 'PROJETO_ATIVIDADE']].itertuples(index=False))
          df['fonte'] = df['fonte'].map(vinc_aux)
          df.rename(columns={'VALOR': 'valor'}, inplace=True)
          df = df[['empresa', 'unidorc', 'projativ', 'natureza_despesa', 'regprevgeral', 'fonte', 'valor']]
          validate_fonte(df)
          self.add_data(df)
          break
    if not self.df.empty:
      self.sigef_value()
      self.font_mask()
    return self.df


  def read_folder_105(self) -> pd.DataFrame:
    folder = pathlib.Path(r'/content/drive/MyDrive/PARAMETRIZAÇÕES FOPAG/modificadores_folha/Poa0105')
    validation = self.Validation(3, 7, 15, 21, '0105', None, None, None, 2)
    for arquivo in folder.iterdir():
      if self.e != 50 and self.validate_file(validation, arquivo):
        df = pd.read_excel(arquivo)
        df.loc[(df['Projeto'] == 7002) & (df['Atividade'] == 2738), 'Atividade'] = 2740
        df.loc[(df['Projeto'] == 7003) & (df['Atividade'] == 2762), 'Atividade'] = 2764
        df_rub4 = df.loc[df['Rubrica'] == 4]
        df_des = pd.DataFrame()
        if self.e == 58:
          df_des = df.loc[(df['Projeto'].isin([7002, 7003])) &
                          (df['Tipo Rubrica'] == 'DES') &
                          (df['Rubrica'].isin(
                              [*range(5,94,1)] + [95, 96] + [*range(98,144,1)] + [*range(145, 400, 1)]
                          ))].copy()

        if not df_des.empty:
          df_des.loc[df_des['Tipo Beneficio'].str.contains('APOSENTADO'), 'situacao'] = 1
          df_des.loc[~df_des['Tipo Beneficio'].str.contains('APOSENTADO'), 'situacao'] = 2
          df_des.drop([
              'Regime', 'Empresa', 'Nome Empresa', 'Tipo Beneficio', 'Tipo Rubrica', 'Complemento', 'Nome Rubrica', 'Regime Juridico', 'Código Contábil'
              ], axis=1, inplace=True)
          df_des.rename(columns={
              'Projeto': 'unidorc', 'Atividade': 'projativ', 'Rubrica': 'rubrica', 'Valor': 'valor'
              }, inplace=True)
          df_des['regprevgeral'] = '2-RPPS'
          df_des['empresa'] = 58
          self.add_data(df_des)
          self.get_natureza_despesa()
          self.get_fonte()
          self.validate_natureza_despesa()
        if not df_rub4.empty:
          df_rub4 = df_rub4[['Empresa', 'Atividade', 'Valor']].rename(columns={'Empresa': 'empresa', 'Atividade': 'projativ', 'Valor': 'valor'})
          df_rub4['regprevgeral'] = '2-RPPS'
          df_rub4['empresa'] = df_rub4['empresa'].map(cache(lambda x: self.dp_empr[x]))
          df_rub4['unidorc'] = df_rub4['projativ'].map(cache(lambda x: self.dp_rub4[x]['unidorc']))
          df_rub4['fonte'] = df_rub4['projativ'].map(cache(lambda x: self.dp_rub4[x]['fonte']))
          df_rub4['natureza_despesa'] = df_rub4['projativ'].map(cache(lambda x: self.dp_rub4[x]['natureza_despesa']))
          df_rub4['projativ'] = df_rub4['projativ'].map(cache(lambda x: self.dp_rub4[x]['projativ']))
          df_rub4 = df_rub4.loc[df_rub4['empresa'] == self.e]
          self.add_data(df_rub4)
        break
    if not self.df.empty:
      self.sigef_value()
      self.font_mask()
    return self.df


  def read_folders_408_508(self):
    folders = [pathlib.Path(r'/content/drive/MyDrive/PARAMETRIZAÇÕES FOPAG/modificadores_folha/Poa0408'),
               pathlib.Path(r'/content/drive/MyDrive/PARAMETRIZAÇÕES FOPAG/modificadores_folha/Poa0508')]
    for folder in folders:
      for arquivo in folder.iterdir():
        validation = {'Poa0408': self.Validation(3, 7, 8, 14, '0408', None, None, None, self.validate_p(arquivo.name)),
                      'Poa0508': self.Validation(3, 7, 8, 14, '0508', None, None, None, self.validate_p(arquivo.name))}
        if self.validate_file(validation[folder.parts[-1]], arquivo):
          df = pd.read_excel(arquivo, converters={'natureza_despesa': str, 'fonte': str})
          df = df.loc[df['empresa'] == self.e]
          self.add_data(df)
          break
    if not self.df.empty:
      self.sigef_value()
      self.font_mask()
    return self.df


  def read_folder_verba_sus(self) -> pd.DataFrame:
    folder = pathlib.Path(r'/content/drive/MyDrive/PARAMETRIZAÇÕES FOPAG/modificadores_folha/Verba_SUS')
    for arquivo in folder.iterdir():
      validation = self.Validation(0, 3, 4, 10, 'Sus', None, None, None, self.validate_p(arquivo.name))
      if self.validate_file(validation, arquivo) and self.e == 50:
        df = pd.read_excel(arquivo, converters={'natureza_despesa': str, 'fonte': str})
        df['empresa'] = self.e
        self.add_data(df)
        break
    if not self.df.empty:
      self.font_mask()
    return self.df


  def read_folder_ace_acs(self) -> pd.DataFrame:
    folder = pathlib.Path(r'/content/drive/MyDrive/PARAMETRIZAÇÕES FOPAG/modificadores_folha/ACE_ACS')
    validation = self.Validation(50, 57, 58, 64, 'ACE_ACS', None, None, None, 2)
    for arquivo in folder.iterdir():
      if self.validate_file(validation, arquivo) and self.e == 50:
        dict_reg = {
              'RGPS': '1-RGPS',
              'RPPS': '2-RPPS'
              }
        df = pd.read_excel(arquivo, converters={'CENTRO_CUSTO': str}).rename(columns={
            'REGIME_PREVIDENCIÁRIO': 'regprevgeral',
            'RUBRICA': 'rubrica',
            'VALOR': 'valor'})
        df['empresa'] = 50
        df['unidorc'] = df['CENTRO_CUSTO'].str[:4].astype(int)
        df['subacao'] = df['CENTRO_CUSTO'].str[-4:].astype(int)
        df['regprevgeral'] = df['regprevgeral'].map(cache(lambda x: dict_reg[x]))
        df['situacao'] = 0
        df['CARGO'] = df['CARGO'].map(cache(lambda x: 'AGENTE COMUNITARIO' if 'COMUNITARIO' in x else 'AGENTE DE COMBATE'))
        self.add_data(df)
        self.get_natureza_despesa()
        self.validate_natureza_despesa()
        self.get_fonte()
        self.df = self.df[['empresa', 'unidorc', 'subacao', 'natureza_despesa', 'fonte', 'CARGO', 'valor']].groupby(
            by=['empresa', 'unidorc', 'subacao', 'natureza_despesa', 'fonte', 'CARGO'], as_index=False).sum()
        break
    if not self.df.empty:
      self.sigef_value()
      self.font_mask()
    return self.df


  def change_df_ace_acs(self, df: pd.DataFrame) -> pd.DataFrame:
    if not df.empty:
      df.loc[df['CARGO'] == 'AGENTE COMUNITARIO', ['natureza_despesa', 'fonte']] = '31901154', '1.604.500.001'
      df.loc[df['CARGO'] != 'AGENTE COMUNITARIO', ['natureza_despesa', 'fonte']] = '31901155', '1.604.502.001'
      self.df = df
      self.group_final_data()
      self.df['naoAlterar'] = 1
    return self.df


  def read_folder_decretos(self) -> pd.DataFrame:
    folder = pathlib.Path(r'/content/drive/MyDrive/PARAMETRIZAÇÕES FOPAG/modificadores_folha/Decretos')
    for arquivo in folder.iterdir():
      validation = self.Validation(0, 7, 13, 19, 'Decreto', None, 8, 12, self.validate_p(arquivo.name))
      if self.validate_file(validation, arquivo) and \
      self.dict_emp[arquivo.name[validation.inicial_idx_empresa:validation.last_idx_empresa]] == self.e:
        df = pd.read_excel(arquivo, converters={'elemento ou natureza de despesa': str, 'fonte': str})
        df['empresa'] = self.dict_emp[arquivo.name[validation.inicial_idx_empresa:validation.last_idx_empresa]]
        self.add_data(df)
        break
    if not self.df.empty:
      self.sigef_value()
      self.font_mask()
    return self.df


  def read_folder_estornos(self) -> pd.DataFrame:
    folder = pathlib.Path(r'/content/drive/MyDrive/PARAMETRIZAÇÕES FOPAG/modificadores_folha/Estornos')
    for arquivo in folder.iterdir():
      validation = self.Validation(0, 8, 14, 20, 'Estornos', None, 9, 13, self.validate_p(arquivo.name))
      if self.validate_file(validation, arquivo) and \
      self.dict_emp[arquivo.name[validation.inicial_idx_empresa:validation.last_idx_empresa]] == self.e:
        df = pd.read_excel(arquivo, converters={'natureza_despesa': str, 'fonte': str})
        df['empresa'] = self.dict_emp[arquivo.name[validation.inicial_idx_empresa:validation.last_idx_empresa]]
        self.add_data(df)
        break
    if not self.df.empty:
      self.sigef_value()
      self.font_mask()
    return self.df


In [ ]:
class APPFormat(Payroll):

  dict_p = dict_p
  dict_m = dict_m
  dict_empresa = dict_empresa
  subject = subject
  email_content = email_content


  def __init__(self, df: pd.DataFrame):
    self.df = df


  def app_format(self) -> pd.DataFrame:
    self.group_final_data()
    self.df = self.df.merge(self.df_cnpj[[
        'unidorc', 'cnpj', 'processo', 'complemento', 'banco', 'agencia',
        'conta', 'bancoorigem', 'agenciaorigem', 'contaorigem'
        ]], on='unidorc', how='left')
    self.change_family_salary_account()
    self.change_columns_format()
    self.insert_app_columns()
    self.insert_history()
    self.insert_origin()
    self.app_columns_orderer()
    return self.df


  def change_family_salary_account(self) -> pd.DataFrame:
    self.df.loc[
        self.df.natureza_despesa.isin(['33900811', '33900809']), ['cnpj', 'banco', 'agencia', 'conta']
        ] = list(self.df_cnpj.loc[self.df_cnpj['unidorc'] == 7002, ['cnpj', 'banco', 'agencia', 'conta']].values)
    return self.df


  def change_columns_format(self) -> pd.DataFrame:
    self.df.unidorc = self.df.unidorc.astype(str).str.zfill(5)
    self.df.subacao = self.df.subacao.astype(str).str.zfill(6)
    self.df.fonte = self.df.fonte.str.replace(r'\D', '', regex=True)
    self.df.valor = self.df.valor.astype(str).str.zfill(15)
    return self.df


  def insert_app_columns(self) -> pd.DataFrame:
    self.df['data'] = str(self.d).zfill(2) + '/' + str(self.m).zfill(2) + '/' + str(self.y)
    self.df['unidade_gestora_sigef'] = str(0).zfill(6)
    self.df['gestao_sigef'] = str(0).zfill(5)
    self.df['tipo_credor'] = 2
    self.df['nota'] = str(0).zfill(12)
    self.df['celula'] = self.df.unidorc
    self.df['instrumento'] = str(0).zfill(4)
    self.df['centralizado'] = 0
    self.df['modalidade'] = 1
    return self.df


  def insert_history(self) -> pd.DataFrame:
    self.df['historico'] = ('FOLHA DE PAGAMENTO ' + self.dict_p[self.p] + ' ' + self.dict_m[self.m] + '/' + str(self.y)).ljust(50)
    return self.df


  def insert_origin(self) -> pd.DataFrame:
    origem = {0: {50: 'FOPAQUIN', 51: 'FOPAQDML', 52: 'FOPAQDMH', 55: 'FOPAQDMA', 58: 'FOPAQPRE', 70: 'FOPAQFAS', 80: 'FOPAQCMP'}, # utilizando o nome da quinzenal, pois não foi cadastrada a origem OUTROS no SIGEF
              1: {50: 'FOPAQUIN', 51: 'FOPAQDML', 52: 'FOPAQDMH', 55: 'FOPAQDMA', 58: 'FOPAQPRE', 70: 'FOPAQFAS', 80: 'FOPAQCMP'},
              2: {50: 'FOLHAPAG', 51: 'FOPADMLU', 52: 'FOPADMHB', 55: 'FOPADMAE', 58: 'FOPAPREV', 70: 'FOPAFASC', 80: 'FOPACMPA'},
              3: {50: 'FOPASUPL', 51: 'FOPASDML', 52: 'FOPASDMH', 55: 'FOPASDMA', 58: 'FOPAQPRE', 70: 'FOPASFAS', 80: 'FOPASCMP'}, # utilizando o nome da quinzenal para o PREVIMPA, pois não foi cadastrada a origem SUPLEMENTAR no SIGEF
              4: {50: 'FOPANATA', 51: 'FOPANDML', 52: 'FOPANDMH', 55: 'FOPANDMA', 58: 'FOPANPRE', 70: 'FOPANFAS', 80: 'FOPANCMP'}}
    self.df['origem'] = origem[self.p][self.e]
    return self.df


  def app_columns_orderer(self) -> pd.DataFrame:
    self.df = self.df[['empresa', 'unidorc', 'data', 'valor', 'cnpj', 'tipo_credor', 'unidade_gestora_sigef', 'gestao_sigef', 'nota', 'celula',
                       'subacao', 'fonte', 'natureza_despesa', 'modalidade', 'instrumento', 'complemento', 'centralizado', 'processo', 'banco',
                       'agencia', 'conta', 'bancoorigem', 'agenciaorigem', 'contaorigem', 'historico', 'origem']]
    return self.df


  def save_app_files(self) -> None:
    folder = pathlib.Path(r'/content/drive/MyDrive/PARAMETRIZAÇÕES FOPAG/APPs')
    last_app_path = r'/content/drive/MyDrive/PARAMETRIZAÇÕES FOPAG/utils/last_app.txt'
    self.clean_folder(folder)
    last_app = self.read_file_txt(last_app_path)
    counter = 1
    if self.t == last_app[3:11]:
      counter += int(last_app[-3:])
    app_name = rf'APP{self.t}.{str(counter).zfill(3)}'
    self.df.drop('empresa', axis=1).to_csv(folder / app_name, sep=';', index=False, header=False, lineterminator='\r\n')
    self.write_file_txt(last_app_path, app_name)
    print('Arquivos APPs salvos na pasta APPs.')


  def clean_folder(self, folder: pathlib.PosixPath) -> None:
    for arquivo in folder.iterdir():
      if arquivo.is_dir():
        arquivo.rmdir()
      else:
        pathlib.Path.unlink(arquivo)


  def read_file_txt(self, file_path: str) -> str:
    f = open(file_path, 'r')
    file_content = f.read()
    f.close()
    return file_content


  def write_file_txt(self, file_path: str, content: str) -> None:
    f = open(file_path, 'w')
    f.write(content)
    f.close()


  def revert_app_format(self) -> pd.DataFrame:
    self.df[
        ['unidorc', 'valor', 'unidade_gestora_sigef', 'gestao_sigef', 'nota', 'celula', 'subacao', 'instrumento', 'complemento', 'processo', 'agencia']
        ] = self.df[
        ['unidorc', 'valor', 'unidade_gestora_sigef', 'gestao_sigef', 'nota', 'celula', 'subacao', 'instrumento', 'complemento', 'processo', 'agencia']
        ].astype(int)
    self.revert_sigef_value()
    self.process_mask()
    self.font_mask()
    return self.df


  def process_mask(self) -> pd.DataFrame:
    self.df['processo'] = self.df['processo'].astype(str)
    self.df.loc[self.df.processo.str.len() == 14, 'processo'] = \
      self.df.processo.str[:2] + '.' + self.df.processo.str[2:4] + '.' + \
      self.df.processo.str[4:13] + '-' + self.df.processo.str[13]
    self.df.loc[self.df.processo.str.len() == 13, 'processo'] = \
      self.df.processo.str[:2] + '.' + self.df.processo.str[2] + '.' + \
      self.df.processo.str[3:12] + '-' + self.df.processo.str[12]
    return self.df


  def generate_excel_file(self) -> None:
    file_name = rf'{str(self.y)+str(self.m).zfill(2)}p{self.p}_{self.dict_empresa[self.e]}_previa.xlsx'
    self.revert_app_format()
    if self.e != 58:
      self.df.drop('empresa', axis=1).to_excel(file_name, index=False)
    else:
      available = ResourceAvailability().read_resource_availability()
      df = self.group_by_element(self.df.copy())
      df = df.merge(available, on=['empresa', 'unidorc', 'subacao', 'elemento', 'fonte'], how='left')
      df.fillna(0, inplace=True)
      df['disponivel - valor'] = df['disponivel'] - df['valor']
      df = df.loc[df['disponivel - valor'] < 0]
      with pd.ExcelWriter(file_name) as writer:
        for i in self.df.unidorc.unique():
          self.df.loc[self.df.unidorc == i].drop('empresa', axis=1).to_excel(writer, sheet_name=str(i), index=False)
        if not df.empty:
          df['observacao'] = 'SOLICITAR DECRETO DE SUPLEMENTACAO, POIS VALOR DISPONÍVEL É INFERIOR AO VALOR EMPENHADO'
          df.drop('empresa', axis=1).to_excel(writer, sheet_name='AVISOS', index=False)


  def send_mail_with_excel(self, recipient_email: list, excel_file: str) -> None:
    msg = EmailMessage()
    SENDER_EMAIL = %env $SENDER_EMAIL
    APP_PASSWORD = %env $APP_PASSWORD
    msg['Subject'] = self.subject
    msg['From'] = SENDER_EMAIL
    msg['To'] = recipient_email
    msg.set_content(self.email_content)
    with open(excel_file, 'rb') as f:
        file_data = f.read()
    msg.add_attachment(file_data, maintype="application", subtype="xlsx", filename=excel_file)
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        smtp.login(SENDER_EMAIL, APP_PASSWORD)
        smtp.send_message(msg)
    print('Email enviado para:', *recipient_email)


In [ ]:
class ResourceAvailability(APPFormat):


  def __init__(self):
    self.df = pd.DataFrame()


  def read_resource_availability(self) -> pd.DataFrame:
    folder = pathlib.Path(rf'/content/drive/MyDrive/PARAMETRIZAÇÕES FOPAG/disponibilidades/{self.dict_empresa[self.e]}')
    for arquivo in folder.iterdir():
      fonte = read_pdf(arquivo, area=[115, 71, 115+20, 71+245], pages=1, pandas_options={'header': None})[0]
      fonte = re.search(r'[\d\.]*', fonte.iloc[0,1]).group()
      df = read_pdf(arquivo, area=[180, 71, 180+370, 71+755], guess=False, pandas_options={'header': [0]}, pages='all', multiple_tables=False, encoding='latin1')[0]
      if fonte and not df.empty:
        df = self.clean_data_resource_availability(df)
        df['unidorc'] = int(re.sub(r'\D', '', df.iloc[1,0]))
        df = self.set_subacao(df)
        df['fonte'] = fonte
        self.df = pd.concat([self.df, df], ignore_index=True)
    if not self.df.empty:
      self.df['empresa'] = self.e
      self.set_elemento()
      self.resource_availability_format()
    return self.df


  def clean_data_resource_availability(self, df: pd.DataFrame) -> pd.DataFrame:

    def str_to_float(value: str) -> float:
      value = str(value).replace('.', '').replace(',', '.')
      value = re.sub(r'[A-Za-z]|\s', '', value)
      return float(0) if value == '' else float(value)

    df.drop(df.loc[df['Agrupamento']=='Agrupamento'].index, inplace=True)
    df_out = df.loc[df['%'].isna()]
    for i in df_out.index:
      df.loc[i-1, 'Agrupamento'] += ' ' + df.loc[i, 'Agrupamento']
    df.drop(df_out.index, inplace=True)
    if len(df.loc[df['Agrupamento']=='Total']) > 1:
      df.drop(df.loc[df['Agrupamento']=='Total'].index[1:], inplace=True)
    df.reset_index(drop=True, inplace=True)
    df.iloc[:, 1:] = df.iloc[:, 1:].map(str_to_float)
    return df


  def set_subacao(self, df: pd.DataFrame) -> pd.DataFrame:
    li = list(df.loc[df['Agrupamento'].str.startswith('00')].index)
    li.append(df.index[-1])
    for i,j in enumerate(li):
      subacao = int(re.sub(r'\D', '',df.loc[j, 'Agrupamento']))
      if i+1 < len(li):
        idx = li[i]
        while idx <= li[i+1]:
          df.loc[idx, 'subacao'] = subacao
          idx += 1
    return df


  def set_elemento(self) -> pd.DataFrame:
    self.df.loc[~self.df['Agrupamento'].str.startswith(('0', 'T')), 'elemento'] = self.df['Agrupamento'].str.replace(r'\D', '', regex=True)
    return self.df


  def resource_availability_format(self) -> pd.DataFrame:
    self.df.dropna(subset='elemento', inplace=True)
    self.df = self.df[['empresa', 'unidorc', 'subacao', 'elemento', 'fonte', 'Disponível']].rename(columns={'Disponível': 'disponivel'})
    return self.df


### Execução do código declarado

In [ ]:
payroll = ReadErgonAPI()
payroll.read_ergon_api()
payroll.clean_data_ergon_api()
payroll = payroll.select_budget_data()
payroll = Payroll(payroll)
payroll.get_natureza_despesa()
payroll.validate_unidorc()
payroll.validate_natureza_despesa()
payroll.get_fonte()
maternity_salary_and_health_leave = ReadModifier().read_folders_1111_1112()
payroll.add_data(maternity_salary_and_health_leave)
family_salary_and_benefits_launched_as_discount = ReadModifier().read_folder_105()
payroll.add_data(family_salary_and_benefits_launched_as_discount)
payroll.get_subacao()
payroll.group_previous_data()
maternity_salary_discount_and_metropolitan_transport = ReadModifier().read_folders_408_508()
payroll.permissive_discount(maternity_salary_discount_and_metropolitan_transport)
sus_budget = ReadModifier().read_folder_verba_sus()
payroll.change_fonte_verba_sus(sus_budget)
payroll.discount_negative_values()
health_agents = ReadModifier().read_folder_ace_acs()
payroll.restrictive_discount(health_agents)
health_agents = ReadModifier().change_df_ace_acs(health_agents)
payroll.add_data(health_agents)
decree = ReadModifier().read_folder_decretos()
payroll.implement_decree(decree)
chargebacks = ReadModifier().read_folder_estornos()
payroll.restrictive_discount(chargebacks)
payroll = APPFormat(payroll.df)
payroll.app_format()
payroll.save_app_files()
payroll.generate_excel_file()
payroll.send_mail_with_excel(user, rf'{str(y)+str(m).zfill(2)}p{p}_{dict_empresa[e]}_previa.xlsx')